In [1]:
### import libraries ###
#from astropy.io import fits
#from astropy import table
import os
import pandas as pd
from func_stat import statResid
import matplotlib.pyplot as plt
#import numpy as np

In [4]:
dirCatalogs='/mnt/Data/Work/Sources/COSMOS/CatalogsFromWorkingPC/ex018/datasets'
fileNameTest='05_COSMOS_SpectrZ_Zlim_1p2_Test_SOMinfo.csv'
fileNameTrain='05_COSMOS_SpectrZ_Zlim_1p2_SOMinfo.csv'

In [12]:
train=pd.read_csv(os.path.join(dirCatalogs,fileNameTrain))
test=pd.read_csv(os.path.join(dirCatalogs,fileNameTest))

In [13]:
from sklearn.ensemble import IsolationForest

In [16]:
X_train=train[train['resid_ML']<=0.15]
X_train=X_train[['Ksmagap3', 'Ymagap3', 'Hmagap3',
       'Jmagap3', 'Bmagap3', 'Vmagap3', 'ipmagap3', 'rmagap3', 'umagap3',
       'zppmagap3','specZ']]

In [20]:
X_test=test[test['resid_ML']<=0.15]
X_test=X_test[['Ksmagap3', 'Ymagap3', 'Hmagap3',
       'Jmagap3', 'Bmagap3', 'Vmagap3', 'ipmagap3', 'rmagap3', 'umagap3',
       'zppmagap3','specZ']]
X_outliers=test[test['resid_ML']>0.15]
X_outliers=X_outliers[['Ksmagap3', 'Ymagap3', 'Hmagap3',
       'Jmagap3', 'Bmagap3', 'Vmagap3', 'ipmagap3', 'rmagap3', 'umagap3',
       'zppmagap3','specZ']]

In [21]:
# training the model
clf = IsolationForest(max_samples=1000)
clf.fit(X_train)

# predictions
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_outliers = clf.predict(X_outliers)

/home/alex/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/home/alex/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/home/alex/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/home/alex/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/home/alex/anaconda3/lib/py

In [22]:
print("Accuracy:", list(y_pred_outliers).count(-1)/y_pred_outliers.shape[0])

Accuracy: 0.3333333333333333


In [23]:
y_pred_outliers

array([ 1,  1, -1, -1,  1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1, -1, -1,
       -1,  1,  1,  1, -1,  1, -1,  1,  1, -1,  1,  1,  1])

In [25]:
### Against objects with big SpecScatter

In [26]:
### Working directories and file names ###
dirCatalogs=r'/mnt/Data/Work/Sources/COSMOS/Catalogs/DuplicateCheck'
os.chdir(dirCatalogs)

testDuplInFileName=r"Test_MSpecOrig_duplicatesIn.csv"

In [27]:
test=pd.read_csv(testDuplInFileName)
# INTify Seq column (aka ID)
test['Seq']=test['Seq'].apply(int)
# Fix 'Infinities'
test.loc[test['specZ_outlCoeff']=='Infinity','specZ_outlCoeff']=-99.9
test['specZ_outlCoeff']=test['specZ_outlCoeff'].astype(float)
# Limit crazy spec-z
test=test[(test['z_spec']>0) & (test['z_spec']<8)]

In [28]:
# Creating a column where we calculate max difference between the measurements for every object
test['SpecScatter']=0
test['NumMeasurements']=0
# Identifying and counting objects with multiple measurements
test_uniq=test['Seq'].value_counts()
# Calculating max difference between the measurements for every object 
for i in test_uniq[test_uniq>1].index:
    diff=test.loc[test['Seq']==i,'z_spec'].max()-test.loc[test['Seq']==i,'z_spec'].min()
    test.loc[test['Seq']==i,'SpecScatter']=diff
    test.loc[test['Seq']==i,'NumMeasurements']=test_uniq[i]

In [ ]:
X_test=test[test['resid_ML']<=0.15]
X_test=X_test[['Ksmagap3', 'Ymagap3', 'Hmagap3',
       'Jmagap3', 'Bmagap3', 'Vmagap3', 'ipmagap3', 'rmagap3', 'umagap3',
       'zppmagap3','specZ']]
X_outliers=test[test['resid_ML']>0.15]
X_outliers=X_outliers[['Ksmagap3', 'Ymagap3', 'Hmagap3',
       'Jmagap3', 'Bmagap3', 'Vmagap3', 'ipmagap3', 'rmagap3', 'umagap3',
       'zppmagap3','specZ']]